In [1]:
import os
import sys
from starter import Starter

# Setup working directory
starter = Starter()
starter.start(lambda: os.chdir(os.path.dirname(os.getcwd())))

# Import all necessary modules
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import json
import time
from typing import Dict, List, Tuple, Any

# Import custom modules
from utils.data_loader import DataLoader
from utils.metrics_calculator import MetricsCalculator
from models.rnn import RNNModelBuilder
from layers.embedding import EmbeddingLayer
from layers.simple_rnn import SimpleRNNLayer
from layers.bidirectional import BidirectionalLayer

# Set random seed for reproducibility
np.random.seed(42)

print("RNN Implementation - Tugas Besar 2 IF3270")
print("=" * 50)

Starter has been initialized.
RNN Implementation - Tugas Besar 2 IF3270


In [2]:
# cell 2 - CORRECTED VERSION
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Bidirectional, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from typing import Dict, Any, Tuple
import os
import numpy as np
import matplotlib.pyplot as plt
import json
import os
import time
from typing import Dict, List, Tuple, Any
from utils.data_loader import DataLoader
from utils.metrics_calculator import MetricsCalculator
from models.rnn import RNNModelBuilder

class KerasRNNTrainer:
    """Keras RNN trainer for comparison with custom implementation"""
    
    def __init__(self, data_loader):
        self.data_loader = data_loader
        
    def create_keras_model(self, config: Dict[str, Any]) -> tf.keras.Model:
        """Create Keras RNN model with given configuration"""
        model = Sequential(name=f"keras_rnn_{config.get('experiment_name', 'model')}")
        
        # Embedding layer
        model.add(Embedding(
            input_dim=self.data_loader.preprocessor.vocab_size,
            output_dim=config['embedding_dim'],
            input_length=self.data_loader.preprocessor.max_length,
            name='embedding'
        ))
        
        # RNN layers
        for i in range(config['num_rnn_layers']):
            return_sequences = i < config['num_rnn_layers'] - 1
            
            rnn_layer = SimpleRNN(
                config['rnn_units'],
                activation=config['activation'],
                return_sequences=return_sequences,
                name=f'simple_rnn_{i}'
            )
            
            if config['bidirectional']:
                model.add(Bidirectional(rnn_layer, name=f'bidirectional_{i}'))
            else:
                model.add(rnn_layer)
            
            # Add dropout after each RNN layer except the last
            if i < config['num_rnn_layers'] - 1:
                model.add(Dropout(config['dropout_rate'], name=f'dropout_{i}'))
        
        # Final dropout and dense layer
        model.add(Dropout(config['dropout_rate'], name='dropout_final'))
        model.add(Dense(self.data_loader.num_classes, activation='softmax', name='dense'))
        
        # Compile model
        model.compile(
            optimizer=Adam(learning_rate=0.001),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )
        
        return model
    
    def train_model(self, config: Dict[str, Any], X_train, y_train, X_valid, y_valid, 
                   epochs: int = 10, batch_size: int = 32) -> Tuple[tf.keras.Model, Dict]:
        """Train Keras model and return model + history"""
        
        print(f"Creating and training Keras model...")
        model = self.create_keras_model(config)
        
        print(f"Model summary:")
        model.summary()
        
        # Train model
        history = model.fit(
            X_train, y_train,
            validation_data=(X_valid, y_valid),
            epochs=epochs,
            batch_size=batch_size,
            verbose=1
        )
        
        return model, history.history
    
    def extract_weights_for_custom_model(self, keras_model: tf.keras.Model, config: Dict[str, Any]) -> Dict[str, Dict[str, np.ndarray]]:
        """Extract weights from Keras model in format compatible with custom model"""
        weights_dict = {}
        
        # Create mapping from Keras layer names to custom layer names
        keras_to_custom_mapping = self._create_layer_name_mapping(config)
        
        for layer in keras_model.layers:
            layer_weights = layer.get_weights()
            if len(layer_weights) == 0:
                continue
                
            keras_layer_name = layer.name
            
            # Map to custom layer name
            if keras_layer_name in keras_to_custom_mapping:
                custom_layer_name = keras_to_custom_mapping[keras_layer_name]
            else:
                print(f"Warning: No mapping found for Keras layer '{keras_layer_name}'")
                continue
            
            if 'embedding' in keras_layer_name:
                weights_dict[custom_layer_name] = {
                    'embedding_matrix': layer_weights[0]
                }
            elif 'simple_rnn' in keras_layer_name:
                # SimpleRNN weights: [W_input, W_recurrent, bias]
                # Keras: input @ W_input + hidden @ W_recurrent + bias
                # Custom: input @ W_ih.T + hidden @ W_hh.T + bias
                # So: W_ih = W_input.T, W_hh = W_recurrent.T
                weights_dict[custom_layer_name] = {
                    'W_ih': layer_weights[0].T,  # (input_size, hidden_size) -> (hidden_size, input_size)
                    'W_hh': layer_weights[1].T,  # (hidden_size, hidden_size) -> (hidden_size, hidden_size)
                    'b_h': layer_weights[2]      # (hidden_size,)
                }
            elif 'bidirectional' in keras_layer_name:
                # Bidirectional layer weights: forward + backward
                # Keras stores as [forward_W_input, forward_W_recurrent, forward_bias, backward_W_input, backward_W_recurrent, backward_bias]
                if len(layer_weights) >= 6:
                    weights_dict[custom_layer_name] = {
                        'forward_W_ih': layer_weights[0].T,
                        'forward_W_hh': layer_weights[1].T,
                        'forward_b_h': layer_weights[2],
                        'backward_W_ih': layer_weights[3].T,
                        'backward_W_hh': layer_weights[4].T,
                        'backward_b_h': layer_weights[5]
                    }
            elif 'dense' in keras_layer_name:
                # Dense layer weights: [W, b]
                # Keras: input @ W + b  where W is (input_size, output_size)
                # Custom: input @ W.T + b where W is (output_size, input_size)
                # So: W_custom = W_keras.T
                weights_dict[custom_layer_name] = {
                    'W': layer_weights[0].T,  # (input_size, output_size) -> (output_size, input_size)
                    'b': layer_weights[1]     # (output_size,)
                }
        
        return weights_dict
    
    def _create_layer_name_mapping(self, config: Dict[str, Any]) -> Dict[str, str]:
        """Create mapping from Keras layer names to custom layer names"""
        mapping = {
            'embedding': 'embedding',
            'dropout_final': 'dropout_final',
            'dense': 'classification'
        }
        
        # Add RNN layer mappings
        for i in range(config['num_rnn_layers']):
            if config['bidirectional']:
                mapping[f'bidirectional_{i}'] = f'bidirectional_rnn_{i}'
            else:
                mapping[f'simple_rnn_{i}'] = f'rnn_{i}'
            
            # Add dropout mappings for intermediate layers
            if i < config['num_rnn_layers'] - 1:
                mapping[f'dropout_{i}'] = f'dropout_{i}'
        
        return mapping
    
    def save_keras_weights(self, keras_model: tf.keras.Model, config: Dict[str, Any], filepath: str) -> Dict[str, Dict[str, np.ndarray]]:
        """Save Keras weights to file and return weights dict"""
        weights_dict = self.extract_weights_for_custom_model(keras_model, config)
        
        # Flatten weights dict for saving
        flattened_weights = {}
        for layer_name, layer_weights in weights_dict.items():
            for weight_name, weight_value in layer_weights.items():
                flattened_weights[f"{layer_name}_{weight_name}"] = weight_value
        
        np.savez(filepath, **flattened_weights)
        print(f"Saved custom weights to {filepath}")
        print(f"Layer mapping: {list(weights_dict.keys())}")
        
        return weights_dict


class RNNExperimentRunner:
    """RNN experiment runner that handles data loader setup correctly"""
    
    def __init__(self, results_dir: str = "results"):
        self.results_dir = results_dir
        self.data_loader = None
        self.keras_trainer = None
        
        # Create results directory
        os.makedirs(results_dir, exist_ok=True)
        
        # Store data
        self.X_train = None
        self.y_train = None
        self.X_valid = None
        self.y_valid = None
        self.X_test = None
        self.y_test = None
        
        # Experiment results
        self.experiment_results = {}
    
    def setup_data(self, data_loader, X_train, y_train, X_valid, y_valid, X_test, y_test):
        """Setup the experiment runner with prepared data"""
        self.data_loader = data_loader
        self.keras_trainer = KerasRNNTrainer(data_loader)
        
        self.X_train = X_train
        self.y_train = y_train
        self.X_valid = X_valid
        self.y_valid = y_valid
        self.X_test = X_test
        self.y_test = y_test
        
        print(f"Data setup complete:")
        print(f"  Vocabulary size: {self.data_loader.preprocessor.vocab_size}")
        print(f"  Number of classes: {self.data_loader.num_classes}")
        print(f"  Training samples: {len(X_train)}")
    
    def run_single_experiment(self, config: Dict[str, Any], experiment_name: str, 
                            epochs: int = 15) -> Dict[str, Any]:
        """Run single experiment with Keras training and custom implementation comparison"""
        
        if self.data_loader is None:
            raise ValueError("Data not set up. Call setup_data() first.")
        
        print(f"\n{'='*60}")
        print(f"Running experiment: {experiment_name}")
        print(f"Configuration: {config}")
        print(f"{'='*60}")
        
        start_time = time.time()
        
        # Step 1: Train Keras model
        print("Step 1: Training Keras model...")
        keras_model, training_history = self.keras_trainer.train_model(
            config, self.X_train, self.y_train, self.X_valid, self.y_valid, 
            epochs=epochs
        )
        
        # Step 2: Evaluate Keras model
        print("Step 2: Evaluating Keras model...")
        keras_test_loss, keras_test_acc = keras_model.evaluate(self.X_test, self.y_test, verbose=0)
        keras_predictions = keras_model.predict(self.X_test, verbose=0)
        keras_pred_labels = np.argmax(keras_predictions, axis=1)
        keras_f1 = MetricsCalculator.macro_f1_score(self.y_test, keras_pred_labels, self.data_loader.num_classes)
        
        print(f"Keras Results - Accuracy: {keras_test_acc:.4f}, F1: {keras_f1:.4f}")
        
        # Step 3: Save Keras weights and extract for custom model
        keras_weights_path = os.path.join(self.results_dir, f"{experiment_name}_keras.weights.h5")
        keras_model.save_weights(keras_weights_path)
        
        custom_weights_path = os.path.join(self.results_dir, f"{experiment_name}_custom_weights.npz")
        weights_dict = self.keras_trainer.save_keras_weights(keras_model, config, custom_weights_path)
        
        # Step 4: Create custom model and load weights
        print("Step 3: Creating custom model and loading weights...")
        custom_model = RNNModelBuilder.create_simple_rnn_model(
            vocab_size=self.data_loader.preprocessor.vocab_size,
            num_classes=self.data_loader.num_classes,
            **config
        )
        
        # Load weights into custom model
        print("Loading weights into custom model...")
        custom_model.set_weights(weights_dict)
        
        # Verify weights were loaded correctly
        custom_weights = custom_model.get_weights()
        print(f"Custom model layers with weights: {list(custom_weights.keys())}")
        
        # Step 5: Evaluate custom model
        print("Step 4: Evaluating custom model...")
        
        # Test on a small subset first to debug
        test_subset = self.X_test[:5]
        keras_subset_pred = keras_model.predict(test_subset, verbose=0)
        custom_subset_pred = custom_model.predict(test_subset)
        
        print(f"Sample predictions comparison (first 2 samples):")
        print(f"Keras  : {keras_subset_pred[:2]}")
        print(f"Custom : {custom_subset_pred[:2]}")
        print(f"Difference: {np.max(np.abs(keras_subset_pred[:2] - custom_subset_pred[:2]))}")
        
        # Full evaluation
        custom_predictions = custom_model.predict(self.X_test)
        custom_pred_labels = np.argmax(custom_predictions, axis=1)
        custom_accuracy = MetricsCalculator.accuracy(self.y_test, custom_pred_labels)
        custom_f1 = MetricsCalculator.macro_f1_score(self.y_test, custom_pred_labels, self.data_loader.num_classes)
        
        print(f"Custom Results - Accuracy: {custom_accuracy:.4f}, F1: {custom_f1:.4f}")
        
        # Step 6: Compare predictions
        prediction_similarity = np.mean(np.isclose(keras_predictions, custom_predictions, atol=1e-3))
        max_prediction_diff = np.max(np.abs(keras_predictions - custom_predictions))
        
        print(f"Comparison - Similarity: {prediction_similarity:.4f}, Max Diff: {max_prediction_diff:.6f}")
        
        end_time = time.time()
        
        # Prepare results
        results = {
            'experiment_name': experiment_name,
            'config': config,
            'keras_metrics': {
                'test_accuracy': float(keras_test_acc),
                'test_f1': float(keras_f1),
                'test_loss': float(keras_test_loss)
            },
            'custom_metrics': {
                'test_accuracy': float(custom_accuracy),
                'test_f1': float(custom_f1)
            },
            'comparison': {
                'prediction_similarity': float(prediction_similarity),
                'max_prediction_difference': float(max_prediction_diff),
                'f1_difference': float(abs(keras_f1 - custom_f1)),
                'accuracy_difference': float(abs(keras_test_acc - custom_accuracy))
            },
            'training_history': training_history,
            'execution_time': end_time - start_time,
            'model_summary': custom_model.summary(),
            'keras_weights_path': keras_weights_path,
            'custom_weights_path': custom_weights_path
        }
        
        return results

    def run_multiple_experiments(self, experiment_configs: Dict[str, List[Dict[str, Any]]], 
                               epochs: int = 15) -> Dict[str, List[Dict[str, Any]]]:
        """Run multiple experiments for hyperparameter analysis"""
        
        all_results = {}
        
        for experiment_type, configs in experiment_configs.items():
            print(f"\n{'='*80}")
            print(f"Running {experiment_type} experiments")
            print(f"{'='*80}")
            
            results = []
            
            for i, config in enumerate(configs):
                experiment_name = f"{experiment_type}_{i}"
                result = self.run_single_experiment(config, experiment_name, epochs)
                results.append(result)
                
                # Save intermediate results
                self._save_results({experiment_type: results})
            
            all_results[experiment_type] = results
        
        # Save final results
        self._save_results(all_results)
        
        return all_results
    
    def _save_results(self, results: Dict[str, List[Dict[str, Any]]]):
        """Save experiment results to JSON file"""
        results_path = os.path.join(self.results_dir, "experiment_results.json")
        
        # Convert numpy types to native Python types for JSON serialization
        serializable_results = {}
        for exp_type, exp_results in results.items():
            serializable_results[exp_type] = []
            for result in exp_results:
                serializable_result = {}
                for key, value in result.items():
                    if key == 'training_history':
                        # Convert training history numpy arrays to lists
                        serializable_result[key] = {
                            k: [float(x) for x in v] if isinstance(v, (list, np.ndarray)) else v
                            for k, v in value.items()
                        }
                    elif isinstance(value, np.ndarray):
                        serializable_result[key] = value.tolist()
                    elif isinstance(value, dict):
                        serializable_result[key] = {
                            k: float(v) if isinstance(v, (np.floating, np.integer)) else v
                            for k, v in value.items()
                        }
                    else:
                        serializable_result[key] = value
                serializable_results[exp_type].append(serializable_result)
        
        with open(results_path, 'w') as f:
            json.dump(serializable_results, f, indent=2)
        
        print(f"Results saved to {results_path}")

2025-05-28 19:16:43.906705: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-28 19:16:43.907657: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-28 19:16:43.910209: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-28 19:16:43.917259: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748434603.929850   77767 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748434603.93

In [3]:
print("\n1. Loading and preprocessing data...")

# Initialize data loader
data_dir = "../data"  # Adjust path as needed
data_loader = DataLoader(data_dir)

# Check if data directory exists and create sample data if needed
if not os.path.exists(os.path.join(data_dir, 'nusax')):
    print("Warning: NusaX dataset not found in data/nusax/")
    print("Please ensure you have the following files:")
    print("  - data/nusax/train.csv")
    print("  - data/nusax/valid.csv") 
    print("  - data/nusax/test.csv")
    print("\nUsing existing sample data from paste.txt...")
    
# Load and prepare data
try:
    X_train, y_train, X_valid, y_valid, X_test, y_test = data_loader.prepare_data(
        max_vocab_size=5000,
        max_length=50,
        min_freq=1
    )
    
    print(f"Data loaded successfully!")
    print(f"Vocabulary size: {data_loader.preprocessor.vocab_size}")
    print(f"Number of classes: {data_loader.num_classes}")
    print(f"Max sequence length: {data_loader.preprocessor.max_length}")
    
    # Show class distribution
    unique, counts = np.unique(y_train, return_counts=True)
    print(f"\nClass distribution in training data:")
    for class_id, count in zip(unique, counts):
        class_name = data_loader.reverse_label_encoder[class_id]
        print(f"  {class_name}: {count} ({count/len(y_train)*100:.1f}%)")
        
    print(f"\nData shapes:")
    print(f"  X_train: {X_train.shape}")
    print(f"  y_train: {y_train.shape}")
    print(f"  X_valid: {X_valid.shape}")
    print(f"  y_valid: {y_valid.shape}")
    print(f"  X_test: {X_test.shape}")
    print(f"  y_test: {y_test.shape}")
        
except Exception as e:
    print(f"Error loading data: {e}")
    raise e

# Test basic model components
print("\n2. Testing RNN model components...")

# Test embedding layer
print("Testing Embedding Layer...")
embedding_layer = EmbeddingLayer(vocab_size=data_loader.preprocessor.vocab_size, embedding_dim=64)
test_tokens = X_train[:2]  # Take first 2 samples
embedded = embedding_layer.forward(test_tokens)
print(f"  Embedding test - Input: {test_tokens.shape}, Output: {embedded.shape}")

# Test RNN layer
print("Testing Simple RNN Layer...")
rnn_layer = SimpleRNNLayer(hidden_size=32, return_sequences=False)
rnn_output = rnn_layer.forward(embedded)
print(f"  RNN test - Input: {embedded.shape}, Output: {rnn_output.shape}")

# Test bidirectional RNN
print("Testing Bidirectional RNN Layer...")
bidirectional_rnn = BidirectionalLayer(
    SimpleRNNLayer,
    hidden_size=32,
    return_sequences=False
)
bi_output = bidirectional_rnn.forward(embedded)
print(f"  Bidirectional RNN test - Input: {embedded.shape}, Output: {bi_output.shape}")

print("✓ All components working correctly!")


1. Loading and preprocessing data...
Loaded data:
  Train: 500 samples
  Valid: 100 samples
  Test: 100 samples
Built vocabulary with 42 words
Most frequent words: ['ini', 'yang', 'sangat', 'dengan', 'produk', 'biasa', 'kualitas', 'untuk', 'saya', 'senang']
Label encoding:
  negative: 0
  neutral: 1
  positive: 2

Data shapes:
  X_train: (500, 50)
  y_train: (500,)
  X_valid: (100, 50)
  y_valid: (100,)
  X_test: (100, 50)
  y_test: (100,)
Data loaded successfully!
Vocabulary size: 42
Number of classes: 3
Max sequence length: 50

Class distribution in training data:
  negative: 150 (30.0%)
  neutral: 150 (30.0%)
  positive: 200 (40.0%)

Data shapes:
  X_train: (500, 50)
  y_train: (500,)
  X_valid: (100, 50)
  y_valid: (100,)
  X_test: (100, 50)
  y_test: (100,)

2. Testing RNN model components...
Testing Embedding Layer...
  Embedding test - Input: (2, 50), Output: (2, 50, 64)
Testing Simple RNN Layer...
  RNN test - Input: (2, 50, 64), Output: (2, 32)
Testing Bidirectional RNN Layer

In [4]:
# cell 4 - CORRECTED VERSION
print("\n3. Running a single experiment test with FIXED runner...")

# Create FIXED experiment runner
fixed_runner = RNNExperimentRunner("results")

# Setup with prepared data
fixed_runner.setup_data(
    data_loader, X_train, y_train, X_valid, y_valid, X_test, y_test
)

# Test configuration
test_config = {
    'embedding_dim': 64,
    'rnn_units': 32,
    'num_rnn_layers': 1,
    'bidirectional': False,
    'dropout_rate': 0.2,
    'activation': 'tanh'
}

# Run single experiment with fewer epochs for testing
print("Running single test experiment (5 epochs)...")
test_result = fixed_runner.run_single_experiment(
    test_config, 
    "test_experiment_fixed", 
    epochs=5
)

# Display results
print("\n" + "="*50)
print("FIXED EXPERIMENT TEST RESULTS")
print("="*50)
print(f"Experiment: {test_result['experiment_name']}")
print(f"Configuration: {test_result['config']}")
print(f"\nKeras Results:")
print(f"  Accuracy: {test_result['keras_metrics']['test_accuracy']:.4f}")
print(f"  F1 Score: {test_result['keras_metrics']['test_f1']:.4f}")
print(f"  Loss: {test_result['keras_metrics']['test_loss']:.4f}")
print(f"\nCustom Implementation Results:")
print(f"  Accuracy: {test_result['custom_metrics']['test_accuracy']:.4f}")
print(f"  F1 Score: {test_result['custom_metrics']['test_f1']:.4f}")
print(f"\nComparison:")
print(f"  Prediction Similarity: {test_result['comparison']['prediction_similarity']:.4f}")
print(f"  Max Prediction Difference: {test_result['comparison']['max_prediction_difference']:.6f}")
print(f"  F1 Difference: {test_result['comparison']['f1_difference']:.4f}")
print(f"  Accuracy Difference: {test_result['comparison']['accuracy_difference']:.4f}")
print(f"\nExecution Time: {test_result['execution_time']:.2f} seconds")

# Check if implementation is working correctly
if test_result['comparison']['prediction_similarity'] > 0.95:
    print("\n✓ SUCCESS: Custom implementation matches Keras very well!")
    print("Proceeding with full experiments...")
    
    # If successful, run a bidirectional test as well
    print("\n" + "="*50)
    print("TESTING BIDIRECTIONAL RNN...")
    print("="*50)
    
    bidirectional_config = {
        'embedding_dim': 64,
        'rnn_units': 32,
        'num_rnn_layers': 1,
        'bidirectional': True,
        'dropout_rate': 0.2,
        'activation': 'tanh'
    }
    
    bidirectional_result = fixed_runner.run_single_experiment(
        bidirectional_config, 
        "test_bidirectional_fixed", 
        epochs=5
    )
    
    print(f"\nBidirectional Test Results:")
    print(f"  Prediction Similarity: {bidirectional_result['comparison']['prediction_similarity']:.4f}")
    print(f"  Max Prediction Difference: {bidirectional_result['comparison']['max_prediction_difference']:.6f}")
    
    if bidirectional_result['comparison']['prediction_similarity'] > 0.95:
        print("✓ Bidirectional RNN also working correctly!")
    else:
        print("⚠ Bidirectional RNN needs more debugging")
        
else:
    print(f"\n⚠ WARNING: Still low similarity ({test_result['comparison']['prediction_similarity']:.4f})")
    print("Additional debugging needed...")
    
    # Print more detailed debugging info
    print("\nDEBUGGING INFO:")
    print("="*30)
    
    # Check if weights were loaded correctly
    if 'custom_weights_path' in test_result:
        try:
            loaded_weights = np.load(test_result['custom_weights_path'])
            print(f"Saved weights keys: {list(loaded_weights.keys())}")
        except:
            print("Could not load saved weights file")
    
    # Check model structure
    print(f"\nModel Summary:")
    print(test_result['model_summary'])


3. Running a single experiment test with FIXED runner...
Data setup complete:
  Vocabulary size: 42
  Number of classes: 3
  Training samples: 500
Running single test experiment (5 epochs)...

Running experiment: test_experiment_fixed
Configuration: {'embedding_dim': 64, 'rnn_units': 32, 'num_rnn_layers': 1, 'bidirectional': False, 'dropout_rate': 0.2, 'activation': 'tanh'}
Step 1: Training Keras model...
Creating and training Keras model...
Model summary:


/home/ganadipa/code/kuliah/sem6/Machine-learning/Tugas-Besar-II-ML/.venv/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
2025-05-28 19:16:45.110258: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "keras_rnn_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_0 (SimpleRNN)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_final (Dropout)         │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.6362 - loss: 0.8648 - val_accuracy: 0.2000 - val_loss: 1.4289
Epoch 2/5
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 1.0000 - loss: 0.2418 - val_accuracy: 0.0000e+00 - val_loss: 1.6726
Epoch 3/5
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 1.0000 - loss: 0.1096 - val_accuracy: 0.0000e+00 - val_loss: 1.9122
Epoch 4/5
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 1.0000 - loss: 0.0612 - val_accuracy: 0.0000e+00 - val_loss: 2.0467
Epoch 5/5
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 1.0000 - loss: 0.0412 - val_accuracy: 0.0000e+00 - val_loss: 2.1704
Step 2: Evaluating Keras model...
Keras Results - Accuracy: 0.4000, F1: 0.4667
Saved custom weights to results/test_experiment_fixed_custom_weights.npz
Layer mapping: ['embedding', 'rnn_0', 'classification']
Step 3: Creating custom model and loading weights...
Loading weights into custom model...
Setting weights for model layers...
Available weight keys: [